In [1]:
import sys; sys.path.append('..'); from cadence.imports import *; import cadence as cd

In [2]:
# !pip install -r ../../prosodic/requirements.txt

In [3]:
import sys; sys.path.append('../../prosodic')
import prosodic as p
p.config['print_to_screen']=False

In [4]:
txt="""In Xanadu did Kubla Khan
A stately pleasure-dome decree:
Where Alph, the sacred river, ran
Through caverns measureless to man
   Down to a sunless sea.
"""
line=txt.split('\n')[0]

In [5]:

def prosodic_line_to_data(line,viols_in_parse_str=True, total_col='*total'):
    for parse_i,parse in enumerate(line.allParses()):
        o=[]
        parse_str=parse.posString(viols=viols_in_parse_str)
        parse_mstr=parse.str_meter()
        for pos_i,pos in enumerate(parse.positions):
            constraintd=dict(sorted(
                (constraint_names_from_prosodic.get(k.name,k.name),v)
                for k,v in pos.constraintScores.items()
            ))
            
            if total_col not in constraintd: constraintd[total_col]=sum(constraintd.values())
            constraintd['*clash']=np.nan
            constraintd['*lapse']=np.nan
            mval=pos.meterVal
            mval2=''.join(mval for n in range(len(pos.slots)))
            nsyll=0
            for slot_i,slot in enumerate(pos.slots):
                nsyll+=1
                odx=dict(
                    parse_i=parse_i,
                    parse=parse_mstr,
                    parse_str=parse_str,
                    parse_pos_i=pos_i,
                    parse_pos=mval2,
                    parse_syll_i=nsyll,
                    parse_syll=mval,
                    
                    **constraintd,
                    is_w=int(mval=='w'),
                    is_s=int(mval=='s'),
                    parse_num_pos=len(parse.positions),
                    parse_num_syll=len(parse_mstr),
                )
                o.append(odx)
        yield pd.DataFrame(o)

def parse_prosodic(txt_or_txtdf, lang='en',
                   meter='default_english', use_espeak=True,
                   constraints=None,constraint_weights=None):
    import prosodic as p
    p.config['resolve_optionality']=0
    p.config['print_to_screen']=True
    p.config['en_TTS_ENGINE']='espeak' if use_espeak else 'none'
    
    meter_obj=p.config['meters'][meter]
    config=default_config=meter_obj.config
    if constraints is not None:
        constraints_pros=[
            f'{constraint_names_in_prosodic[c]}/{constraint_weights[i] if constraint_weights is not None else 1}'
            for c in constraints
        ]
        config['constraints']=constraints_pros
        meter_obj=p.Meter(config)
    
    
    txtdf = scan(txt_or_txtdf) if type(txt_or_txtdf)==str else txt_or_txtdf
    txtdf = txtdf.sort_index()
    
    o=[]
    for combo_i,combodf in sorted(txtdf.groupby('combo_i')):
        combodf=combodf.reset_index()
        words=[]
        for wi,wdf in combodf.groupby('word_i'):
            ipa_str='.'.join(wdf.syll_ipa)
            sylls_text=list(wdf.syll_str)
            token = wdf.iloc[0].word_str
            word_obj = p.dict[lang].make((ipa_str,sylls_text), token)
            wordtoken_obj = p.WordToken([word_obj], token)
            words.append(wordtoken_obj)
        
        l=p.Line()
        l.children = words
        
        l.parse(meter=meter_obj)
        for parsedf in prosodic_line_to_data(l):
            return pd.concat([combodf,parsedf],axis=1)
        


In [6]:
for combodf in iter_combos(scan(line)): pass

In [8]:
odf=parse(line,num_proc=1)

Metrically parsing lines (+word IPA combinations) [x1]: 100%|██████████| 4/4 [00:00<00:00, 38.91it/s]

8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8


In [9]:
odf

*total  ...  prom_weight
stanza_i line_i line_str                 parse_rank combo_i combo_stress combo_ipa                        parse_i parse    parse_str                     parse_pos_i parse_pos word_i word_str word_ipa_i word_ipa   syll_i combo_syll_i syll_str syll_ipa syll_stress syll_weight parse_syll_i parse_syll          ...             
1        1      In Xanadu did Kubla Khan 1          2       PUPUPPUP     'ɪn zæ.'nɑ.dʉː 'dɪd 'kʌ.blʌ 'kɑn 0       swswssws IN|xa|NA|du|DID.KU*|bla|KHAN  0           s         1      In       2          'ɪn        1      0            In       'ɪn      P           H           1            s              0.0  ...            1
                                                                                                                                                         1           w         2      Xanadu   1          zæ.'nɑ.dʉː 1      1            Xa       zæ       U           L           1            w              0.0  ...            0
                                                                                                                                                         2           s         2      Xanadu   1          zæ.'nɑ.dʉː 2      2            na       'nɑ      P           L           1            s              0.0  ...            0
                                                                                                                                                         3           w         2      Xanadu   1          zæ.'nɑ.dʉː 3      3            du       dʉː      U           L           1            w              0.0  ...            0
                                                                                                                                                         4           ss        3      did      1          'dɪd       1      4            did      'dɪd     P           H           1            s              1.0  ...            1
...                                                                                                                                                                                                                                                                                                            ...  ...          ...
                                         15         0       UUPUPPUP     ɪn zæ.'nɑ.dʉː 'dɪd 'kʌ.blʌ 'kɑn  4       wwswswws in.xa*|NA|du|DID|ku.bla*|KHAN 2           w         2      Xanadu   1          zæ.'nɑ.dʉː 3      3            du       dʉː      U           L           1            w              0.0  ...            0
                                                                                                                                                         3           s         3      did      1          'dɪd       1      4            did      'dɪd     P           H           1            s              0.0  ...            1
                                                                                                                                                         4           ww        4      Kubla    1          'kʌ.blʌ    1      5            Ku       'kʌ      P           L           1            w              2.0  ...            0
                                                                                                                                                                                                                     2      6            bla      blʌ      U           L           2            w              2.0  ...            0
                                                                                                                                                         5           s         5      Khan     1          'kɑn       1      7            Khan     'kɑn     P           H           1            s              0.0  ...            1

[120 rows x 24 columns]

In [35]:
# for

In [48]:
pcols1=set(parse_combo(combodf).reset_index().columns)
pcols2=set(parse_prosodic(combodf).columns)
pcols=list(pcols1&pcols2)

8 8


In [50]:
parse_combo(combodf).reset_index()[pcols]

,parse_str,parse_pos_i,syll_ipa,combo_ipa,is_peak,combo_num_syll,combo_stress,word_ipa_i,line_str,is_heavy,parse_num_syll,*clash,is_unstressed,*w-res,combo_syll_i,syll_weight,syll_i,*lapse,syll_stress,is_light,stanza_i,parse_i,prom_weight,parse_pos,combo_i,is_trough,is_stressed,is_w,is_syll,syll_str,is_funcword,parse_syll_i,word_str,parse,*w/peak,word_i,prom_stress,is_s,*f-res,prom_strength,line_i,*w/stressed,parse_num_pos,*s/unstressed,*total,word_ipa,parse_syll
0,in|XA|na|DU|did|KU|bla|KHAN,0,'ɪn,'ɪn zæ.'nɑ.dʉː dɪd 'kʌ.blʌ 'kɑn,0,8,PUPUUPUP,2,In Xanadu did Kubla Khan,1,8,0,0,NaN,0,H,1,0,P,0,1,0,1,w,3,0,1,1,1,In,1,0,In,wswswsws,0,1,1.0,0,NaN,NaN,1,1,8,0,1.0,'ɪn,w
1,in|XA|na|DU|did|KU|bla|KHAN,1,zæ,'ɪn zæ.'nɑ.dʉː dɪd 'kʌ.blʌ 'kɑn,0,8,PUPUUPUP,1,In Xanadu did Kubla Khan,0,8,0,1,NaN,1,L,1,0,U,1,1,0,0,s,3,1,0,0,1,Xa,0,1,Xanadu,wswswsws,0,2,0.0,1,NaN,0.0,1,0,8,1,1.0,zæ.'nɑ.dʉː,s
2,in|XA|na|DU|did|KU|bla|KHAN,2,'nɑ,'ɪn zæ.'nɑ.dʉː dɪd 'kʌ.blʌ 'kɑn,1,8,PUPUUPUP,1,In Xanadu did Kubla Khan,0,8,0,0,NaN,2,L,2,0,P,1,1,0,0,w,3,0,1,1,1,na,0,2,Xanadu,wswswsws,1,2,1.0,0,NaN,1.0,1,1,8,0,2.0,zæ.'nɑ.dʉː,w
3,in|XA|na|DU|did|KU|bla|KHAN,3,dʉː,'ɪn zæ.'nɑ.dʉː dɪd 'kʌ.blʌ 'kɑn,0,8,PUPUUPUP,1,In Xanadu did Kubla Khan,0,8,0,1,NaN,3,L,3,0,U,1,1,0,0,s,3,1,0,0,1,du,0,3,Xanadu,wswswsws,0,2,0.0,1,NaN,0.0,1,0,8,1,1.0,zæ.'nɑ.dʉː,s
4,in|XA|na|DU|did|KU|bla|KHAN,4,dɪd,'ɪn zæ.'nɑ.dʉː dɪd 'kʌ.blʌ 'kɑn,0,8,PUPUUPUP,2,In Xanadu did Kubla Khan,1,8,0,1,NaN,4,H,1,0,U,0,1,0,1,w,3,0,0,1,1,did,1,4,did,wswswsws,0,3,0.0,0,NaN,NaN,1,0,8,0,0.0,dɪd,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,IN.XA|na.du|DID.KU|bla.khan,1,dʉː,'ɪn zæ.'nɑ.dʉː dɪd 'kʌ.blʌ 'kɑn,0,8,PUPUUPUP,1,In Xanadu did Kubla Khan,0,8,0,1,NaN,3,L,3,0,U,1,1,67,0,ww,3,1,0,1,1,du,0,3,Xanadu,sswwssww,0,2,0.0,0,NaN,0.0,1,0,4,0,0.0,zæ.'nɑ.dʉː,w
540,IN.XA|na.du|DID.KU|bla.khan,2,dɪd,'ɪn zæ.'nɑ.dʉː dɪd 'kʌ.blʌ 'kɑn,0,8,PUPUUPUP,2,In Xanadu did Kubla Khan,1,8,0,1,1.0,4,H,1,1,U,0,1,67,1,ss,3,0,0,0,1,did,1,4,did,sswwssww,0,3,0.0,1,1.0,NaN,1,0,4,1,4.0,dɪd,s
541,IN.XA|na.du|DID.KU|bla.khan,2,'kʌ,'ɪn zæ.'nɑ.dʉː dɪd 'kʌ.blʌ 'kɑn,1,8,PUPUUPUP,1,In Xanadu did Kubla Khan,0,8,0,0,1.0,5,L,1,0,P,1,1,67,0,ss,3,0,1,0,1,Ku,0,5,Kubla,sswwssww,0,4,1.0,1,1.0,1.0,1,0,4,0,2.0,'kʌ.blʌ,s
542,IN.XA|na.du|DID.KU|bla.khan,3,blʌ,'ɪn zæ.'nɑ.dʉː dɪd 'kʌ.blʌ 'kɑn,0,8,PUPUUPUP,1,In Xanadu did Kubla Khan,0,8,0,1,NaN,6,L,2,0,U,1,1,67,0,ww,3,1,0,1,1,bla,0,6,Kubla,sswwssww,0,4,0.0,0,NaN,0.0,1,0,4,0,0.0,'kʌ.blʌ,w


In [38]:
# parse_prosodic(combodf) == 

8 8


ValueError: Can only compare identically-labeled DataFrame objects

In [23]:
for combodf in iter_combos(scan(line)): pass
combo_cols=set(combodf.reset_index().columns)
combo_cols
combodf_parsed=parse_combo(combodf).reset_index()
cols_weneed = [c for c in combodf_parsed.columns if c not in combo_cols]
combodf_parsed[cols_weneed]

,parse_i,parse,parse_str,parse_pos_i,parse_pos,parse_syll_i,parse_syll,*total,*clash,*w-res,*lapse,*w/peak,*f-res,*w/stressed,*s/unstressed,is_s,is_w,parse_num_pos,parse_num_syll
0,0,wswswsws,in|XA|na|DU|did|KU|bla|KHAN,0,w,0,w,1.0,0,NaN,0,0,NaN,1,0,0,1,8,8
1,0,wswswsws,in|XA|na|DU|did|KU|bla|KHAN,1,s,1,s,1.0,0,NaN,0,0,NaN,0,1,1,0,8,8
2,0,wswswsws,in|XA|na|DU|did|KU|bla|KHAN,2,w,2,w,2.0,0,NaN,0,1,NaN,1,0,0,1,8,8
3,0,wswswsws,in|XA|na|DU|did|KU|bla|KHAN,3,s,3,s,1.0,0,NaN,0,0,NaN,0,1,1,0,8,8
4,0,wswswsws,in|XA|na|DU|did|KU|bla|KHAN,4,w,4,w,0.0,0,NaN,0,0,NaN,0,0,0,1,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,67,sswwssww,IN.XA|na.du|DID.KU|bla.khan,1,ww,3,w,0.0,0,NaN,0,0,NaN,0,0,0,1,4,8
540,67,sswwssww,IN.XA|na.du|DID.KU|bla.khan,2,ss,4,s,4.0,0,1.0,1,0,1.0,0,1,1,0,4,8
541,67,sswwssww,IN.XA|na.du|DID.KU|bla.khan,2,ss,5,s,2.0,0,1.0,0,0,1.0,0,0,1,0,4,8
542,67,sswwssww,IN.XA|na.du|DID.KU|bla.khan,3,ww,6,w,0.0,0,NaN,0,0,NaN,0,0,0,1,4,8


In [24]:
combo_parsed_pros = parse_prosodic(combodf)

AttributeError: 'str' object has no attribute 'groupby'

In [21]:
combodf_parsed.reset_index()[cols_weneed]

,*clash,*f-res,*lapse,*s/unstressed,*total,*w-res,*w/peak,*w/stressed,is_s,is_w,parse,parse_i,parse_num_pos,parse_num_syll,parse_pos,parse_pos_i,parse_str,parse_syll,parse_syll_i
0,0,NaN,0,0,1.0,NaN,0,1,0,1,wswswsws,0,8,8,w,0,in|XA|na|DU|did|KU|bla|KHAN,w,0
1,0,NaN,0,1,1.0,NaN,0,0,1,0,wswswsws,0,8,8,s,1,in|XA|na|DU|did|KU|bla|KHAN,s,1
2,0,NaN,0,0,2.0,NaN,1,1,0,1,wswswsws,0,8,8,w,2,in|XA|na|DU|did|KU|bla|KHAN,w,2
3,0,NaN,0,1,1.0,NaN,0,0,1,0,wswswsws,0,8,8,s,3,in|XA|na|DU|did|KU|bla|KHAN,s,3
4,0,NaN,0,0,0.0,NaN,0,0,0,1,wswswsws,0,8,8,w,4,in|XA|na|DU|did|KU|bla|KHAN,w,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,0,NaN,0,0,0.0,NaN,0,0,0,1,sswwssww,67,4,8,ww,1,IN.XA|na.du|DID.KU|bla.khan,w,3
540,0,1.0,1,1,4.0,1.0,0,0,1,0,sswwssww,67,4,8,ss,2,IN.XA|na.du|DID.KU|bla.khan,s,4
541,0,1.0,0,0,2.0,1.0,0,0,1,0,sswwssww,67,4,8,ss,2,IN.XA|na.du|DID.KU|bla.khan,s,5
542,0,NaN,0,0,0.0,NaN,0,0,0,1,sswwssww,67,4,8,ww,3,IN.XA|na.du|DID.KU|bla.khan,w,6


In [41]:
cols_weneed=set(parse_cols) - set(combo_cols)
cols_weneed

{'*clash',
 '*f-res',
 '*lapse',
 '*s/unstressed',
 '*total',
 '*w-res',
 '*w/peak',
 '*w/stressed',
 'is_s',
 'is_w',
 'parse',
 'parse_i',
 'parse_num_pos',
 'parse_num_syll',
 'parse_pos',
 'parse_pos_i',
 'parse_rank',
 'parse_str',
 'parse_syll',
 'parse_syll_i'}

In [11]:
l=parse_prosodic('Bird bird bird bird', constraints=['*w/peak','*w/stressed','*s/unstressed','*f-res'])

NameError: name 'parse_prosodic' is not defined

In [43]:
prs=l.bestParse()
pos=prs.positions[0]
prs.constraintScores

{[*strength.w=>-p]: 0.0,
 [*stress.w=>-p]: 2.0,
 [*stress.s=>-u]: 0.0,
 [*footmin-f-resolution]: 0}

In [33]:
for posx in prs.positions: print(posx.constraintScores)

{[*strength.w=>-p]: 0, [*stress.w=>-p]: 0, [*stress.s=>-u]: 0.0}
{[*strength.w=>-p]: 0.0, [*stress.w=>-p]: 0.0, [*stress.s=>-u]: 0}
{[*strength.w=>-p]: 0, [*stress.w=>-p]: 0, [*stress.s=>-u]: 0.0}
{[*strength.w=>-p]: 0.0, [*stress.w=>-p]: 0.0, [*stress.s=>-u]: 0}
{[*strength.w=>-p]: 0, [*stress.w=>-p]: 0, [*stress.s=>-u]: 0.0}
{[*strength.w=>-p]: 0.0, [*stress.w=>-p]: 0.0, [*stress.s=>-u]: 0}


In [10]:
p.config['meters']['default_english'].constraints

[[*strength.w=>-p],
 [*stress.s=>-u],
 [*stress.w=>-p],
 [*footmin-w-resolution],
 [*footmin-f-resolution]]

In [9]:
# p.Meter

Metrically parsing lines (+word IPA combinations) [x8]: 100%|██████████| 4/4 [00:04<00:00,  1.01s/it]


Index(['stanza_i', 'line_i', 'line_str', 'parse_rank', 'combo_i',
       'combo_stress', 'combo_ipa', 'parse_i', 'parse', 'parse_str',
       'parse_pos_i', 'parse_pos', 'word_i', 'word_str', 'word_ipa_i',
       'word_ipa', 'syll_i', 'combo_syll_i', 'syll_str', 'syll_ipa',
       'syll_stress', 'syll_weight', 'parse_syll_i', 'parse_syll', '*total',
       '*w-res', '*f-res', '*w/stressed', '*s/unstressed', '*clash', '*w/peak',
       '*lapse', 'combo_num_syll', 'is_funcword', 'is_heavy', 'is_light',
       'is_peak', 'is_s', 'is_stressed', 'is_syll', 'is_trough',
       'is_unstressed', 'is_w', 'parse_num_pos', 'parse_num_syll',
       'prom_strength', 'prom_stress', 'prom_weight'],
      dtype='object')

In [61]:
def parse_phons():
    output_dict['[*clash_across]']=counts(stress_str,'P#P') + counts(stress_str,'P#S') + counts(stress_str,'S#P') + counts(stress_str,'S#S')
    output_dict['[*clash_within]']=counts(stress_str,'PP') + counts(stress_str,'PS') + counts(stress_str,'SP') + counts(stress_str,'SS')
    output_dict['[*clash_across_primary]']=counts(stress_str,'P#P')
    output_dict['[*clash_within_primary]']=counts(stress_str,'PP')
    output_dict['[*lapse_across]']=counts(stress_str,'U#U')
    output_dict['[*lapse_within]']=counts(stress_str,'UU')
    output_dict['[*WSP]']=0
    output_dict['[*PEAKPROM]']=0
    output_dict['[*High_Stress]']=0
    output_dict['[*Low_Unstress]']=0
    output_dict['[*High_Strong]']=0
    output_dict['[*Low_Weak]']=0
    for s,w,hml,mtr in zip(stress_str,weight_str,sonority_str,meter_str):
        if s=='U' and w=='H':
            output_dict['[*WSP]']+=1
        if (s=='P' or s=='S') and w=='L':
            output_dict['[*PEAKPROM]']+=1
        
        if hml=='H' and s in {'P','S'}:
            output_dict['[*High_Stress]']+=1
        if hml=='L' and s=="U":
            output_dict['[*Low_Unstress]']+=1
        
        if hml=='H' and mtr == 's':
            output_dict['[*High_Strong]']+=1
        if hml=='L' and mtr == 'w':
            output_dict['[*Low_Weak]']+=1